In [1]:
%%javascript
//debugging to see if jupyter-klampt-widget is installed properly
console.log(require.s.contexts._.defined);
var kf = require('nbextensions/klampt/KlamptFrontend');
console.log(kf);

<IPython.core.display.Javascript object>

In [2]:
import time
from klampt import *
from klampt.math import vectorops,so3,se3
from klampt.vis.ipython import KlamptWidget

world = WorldModel()
#world.loadFile("../data/athlete_plane.xml")
world.readFile("../data/robots/tx90l.rob")
kvis = KlamptWidget(world)
#If you'd like to hide the print output, uncomment this line
#clear_output()
display(kvis)

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

***  klampt.vis: using Qt5 as the visualization backend  ***


KlamptWidget(scene={u'object': {u'matrix': [1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 1], u'uuid': u'8c5de…

In [3]:
#testing bug
from klampt.model import ik
from klampt.plan import robotplanning
robot = world.robot(0)
qtemp = [0]*robot.numLinks()
qtemp[3] += 2.0
robot.setConfig(qtemp)
obj = ik.fixed_objective(robot.link(6),local=[0,0,0])
kvis.clearExtras()
kvis.addSphere("ik_target",*obj.getPosition()[1],r=0.02)
kvis.setColor("ik_target",1,0,0)
kvis.addGhost()
kvis.setGhostConfig(qtemp)
kvis.setColor("ghost",0,1,0,0.5)
robot.setConfig([0]*robot.numLinks())
kvis.update()
s = robotplanning.makeSpace(world,robot,equalityConstraints=[obj],movingSubset=[1,2,3])
print s.__class__.__name__
print s.ambientspace.solver.getActiveDofs()

EmbeddedRobotCSpace
[1, 2, 3]


In [9]:
from klampt.math import vectorops
q0 = [0,0,0]
q = s.sampleneighborhood(q0,1.0)
print q,s.feasible(q),s.cspace.feasibilityFailures(q)
robot.setConfig(s.lift(q))
print "error",vectorops.norm(ik.residual(obj))
kvis.update()

solver = ik.solver(obj)
solver.setActiveDofs([1,2,3])
print "Solved?",solver.solve(),solver.getResidual()

[1.917663125283319e-05, -2.3906794360190817, -1.9978456307765398] True []
error 4.87920026665e-05
Solved? True [4.0333376310819347e-05, -1.031504997774102e-05, -2.5445589471151386e-05]


In [ ]:
#add testing

robot = world.robot(0)
q = robot.getConfig()
q[2] += 0.5
q1 = [v for v in q]
q[2] += 0.5
q2 = [v for v in q]
#kvis.add('milestones',[q1,q2])
#kvis.set_color('milestones',0,1,0,0.5)

from klampt.model.trajectory import Trajectory
traj = Trajectory([0,1],[q1,q2])
q1[0] += 0.5
q2[0] += 0.5
kvis.add('milestones',traj)
kvis.setColor('milestones',0,1,0,0.5)

In [ ]:
%%javascript
var w = window.open('', '');
w.document.title = "Screenshot";
var img = new Image();
img.src = $(".p-Widget > div > canvas")[0].toDataURL();
w.document.body.appendChild(img);  